In [21]:
#dependencies and setup
import pandas as pd
#file to load 
file_to_load = "Resources/purchase_data.csv"
#read purchase file
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [22]:
#calculate the number of unique players
players_demographics = purchase_data.loc[:,["Gender","SN","Age"]]
players_demographics = players_demographics.drop_duplicates()
players_demographics
num_players= players_demographics.count()[0]
pd.DataFrame({"Total Players":[num_players]})

,Total Players
0,576


In [23]:
#Run calcualtion to obtain number of unique items average price etc
average_item_price = purchase_data["Price"].mean()
total_purchase_value = purchase_data["Price"].sum()
purchase_count = purchase_data["Price"].count()
item_count = len(purchase_data["Item ID"].unique())

#create a dataframe 
summary_table = pd.DataFrame({
    "Number of Unique Items": item_count,
    "Total Revenue":[total_purchase_value],
    "Number of Purchases":[purchase_count],
    "Average Price":[average_item_price]
})

#minor data munging

summary_table = summary_table.round(2)
# summary_table.head()
summary_table ["Average Price"] = summary_table["Average Price"].map("${:,.2f}".format)
summary_table["Number of Purchases"]= summary_table["Number of Purchases"].map("{:,}".format)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format)
summary_table = summary_table.loc[:,["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]]
#summary Table
summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [24]:
#calculate the number of percentage
gender_demographics_total = players_demographics["Gender"].value_counts()
gender_demographics_percent = gender_demographics_total / num_players
gender_demographics = pd.DataFrame({
    "Total Count": gender_demographics_total,
    "Percentage of Players": gender_demographics_percent
})

#minor data cleaning
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{:,.2%}".format)
gender_demographics

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [25]:
#Run Calculations
gender_purchase_total=purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Price")
gender_average = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_counts = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")

#Calculate normalized purchases
normalized_total = gender_purchase_total/ gender_demographics["Total Count"]

#conver to DataFrame
gender_data = pd.DataFrame({"Purchase Count":gender_counts,
                           "Average Purchase Price": gender_average,
                           "Total Purchase Value":gender_purchase_total,
                           "Normalized Totals": normalized_total

                           })
#cleaning process
gender_data["Average Purchase Price"] = gender_data["Average Purchase Price"].map("${:,.2f}".format)
gender_data["Total Purchase Value"]= gender_data["Total Purchase Value"].map("${:,.2f}".format)
gender_data["Purchase Count"]= gender_data["Purchase Count"].map("{:,}".format)
gender_data["Avg Total Purchase per Person"] = gender_data["Normalized Totals"].map("${:,.2f}".format)
gender_data[["Purchase Count", "Average Purchase Price","Total Purchase Value","Avg Total Purchase per Person"]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [26]:
#Establish bin for ages
age_bins =[0,9.90,14.90,19.90,24.90,29.90,34.90,39.90,9999]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
#categorize the existing playes using bins
players_demographics["Age Ranges"] = pd.cut(players_demographics["Age"], age_bins,
                                           labels=group_names)
age_demographics_totals = players_demographics["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals/num_players
age_demographics = pd.DataFrame({"Total Count": age_demographics_totals,
                                "Percentage of Players": age_demographics_percents})
#clean
age_demographics["Percentage of Players"]=age_demographics["Percentage of Players"].map("{:,.2%}".format)
age_demographics = age_demographics.sort_index()
age_demographics

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [27]:
#bin the purchasing data
purchase_data["Age Ranges"]= pd.cut(purchase_data["Age"],age_bins, labels=group_names)

#calulation time

age_purchase_total = purchase_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Price")
age_average = purchase_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_count = purchase_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

#normalize total

normalized_total = age_purchase_total /age_demographics["Total Count"]

#convert to df dataframe

age_data = pd.DataFrame({
    "Purchase Count": age_count,
    "Average Purchase Price":age_purchase_total,
    "Total Purchase Value":age_purchase_total,
    "Normalized Totals": normalized_total
    
})
age_data["Average Purchase Price"]= age_data["Average Purchase Price"].map("${:,.2f}".format)
age_data["Total Purchase Value"]= age_data["Total Purchase Value"].map("${:,.2f}".format)
age_data["Purchase Count"]= age_data["Purchase Count"].map("${:,.2f}".format)
age_data["Avg Total Purchase per Person"]= age_data["Normalized Totals"].map("${:,.2f}".format)


age_data[["Purchase Count", "Average Purchase Price", "Total Purchase Value","Avg Total Purchase per Person"]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,$23.00,$77.13,$77.13,$4.54
10-14,$28.00,$82.78,$82.78,$3.76
15-19,$136.00,$412.89,$412.89,$3.86
20-24,$365.00,"$1,114.06","$1,114.06",$4.32
25-29,$101.00,$293.00,$293.00,$3.81
30-34,$73.00,$214.00,$214.00,$4.12
35-39,$41.00,$147.67,$147.67,$4.76
40+,$13.00,$38.24,$38.24,$3.19


In [28]:
#top spenders
user_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")

user_average =purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count= purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

user_data =pd.DataFrame({"Total Purchase Value": user_total,
                        "Average Purchase Price": user_average,
                        "Purchase Count":user_count})

user_sorted = user_data.sort_values("Total Purchase Value", ascending=False)
user_sorted["Average Purchase Price"] = user_sorted["Average Purchase Price"].map("${:,.2f}".format)
user_sorted["Total Purchase Value"] = user_sorted["Total Purchase Value"].map("${:,.2f}".format)

user_sorted[["Purchase Count","Average Purchase Price","Total Purchase Value"]].head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [29]:
#Most popular items
item_data = purchase_data[["Item ID","Item Name","Price"]]
#perform calculation
total_item_purchase = item_data.groupby(["Item ID","Item Name"]).sum()["Price"].rename("Total Purchase Price")
average_item_purchase = item_data.groupby(["Item ID","Item Name"]).mean()["Price"]
item_count= item_data.groupby(["Item ID","Item Name"]).count()["Price"].rename("Purchase Count")

#create df

item_data_df = pd.DataFrame({"Total Purchase Value": total_item_purchase,
                            "Item Price": average_item_purchase,
                            "Purchase Count": item_count})

#sort values
item_data_count_sorted = item_data_df.sort_values("Purchase Count", ascending = False)

#cleaning 
item_data_count_sorted["Item Price"] = item_data_count_sorted["Item Price"].map("${:,.2f}".format)
item_data_count_sorted["Purchase Count"] = item_data_count_sorted["Purchase Count"].map("{:,}".format)
item_data_count_sorted["Total Purchase Value"] =  item_data_count_sorted["Total Purchase Value"].map("{:,.2f}".format)

item_data_count_sorted[["Purchase Count", "Item Price", "Total Purchase Value"]].head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,50.76
145,Fiery Glass Crusader,9,$4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,31.77
82,Nirvana,9,$4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,8.16


In [33]:
#item table
item_total_purchase = item_data_df.sort_values("Total Purchase Value", ascending = False)
item_total_purchase["Item Price"] = item_total_purchase["Item Price"].map("${:,.2f}".format)
item_total_purchase["Purchase Count"] = item_total_purchase["Purchase Count"].map("${:,.2f}".format)
item_total_purchase["Total Purchase Value"] = item_total_purchase["Total Purchase Value"].map("${:,.2f}".format)

item_total_purchase[["Purchase Count", "Item Price","Total Purchase Value"]].head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$12.00,$4.23,$50.76
82,Nirvana,$9.00,$4.90,$44.10
145,Fiery Glass Crusader,$9.00,$4.58,$41.22
92,Final Critic,$8.00,$4.88,$39.04
103,Singed Scalpel,$8.00,$4.35,$34.80
